In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [15]:
def replace_column_values(df, column_name, old_value, new_value):
    if column_name in df.columns:
        df[column_name] = df[column_name].replace(old_value, new_value)
        print(f"All instances of '{old_value}' in column '{column_name}' have been changed to '{new_value}'.")
    else:
        print(f"Column '{column_name}' does not exist in the DataFrame.")
    return df

## load dataset for training and testing

In [16]:
train = pd.read_csv('./../train.csv')
train.drop(['Unnamed: 0'], axis=1, inplace=True)
train = replace_column_values(train, 'class', 2 , 0)
test = pd.read_csv('./../test.csv')
test.drop(['Unnamed: 0'], axis=1, inplace=True)
test = replace_column_values(test, 'class', 2 , 0)

All instances of '2' in column 'class' have been changed to '0'.
All instances of '2' in column 'class' have been changed to '0'.


In [33]:
X_train = train.iloc[:, 0:9].values
y_train = train.iloc[:, 9].values
# print(y_train)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

In [23]:
X_test = test.iloc[:, 0:9].values
y_test = test.iloc[:, 9].values

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [91]:
class RiskClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(9, 16)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(16, 32)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(32, 64)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(64, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act_output(self.output(x))
        return x

model = RiskClassifier()
print(model)

# loss function and optimizer
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

RiskClassifier(
  (hidden1): Linear(in_features=9, out_features=16, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=16, out_features=32, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=32, out_features=64, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [92]:
def evaluate(model, X_test, y_test):
  with torch.no_grad():
    y_pred = model(X_test)
  accuracy = (y_pred.round() == y_test).float().mean()
  return accuracy

In [93]:
n_epochs = 120
batch_size = 16


accuracy = evaluate(model, X_test, y_test)
print('accuracy before training {}'.format(accuracy))
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        # print(loss.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # evaluate model on each epoch
    accuracy = evaluate(model, X_test, y_test)
    print(f'Finished epoch {epoch},  accuracy {accuracy}')

accuracy before training 0.34285715222358704
Finished epoch 0,  accuracy 0.34285715222358704
Finished epoch 1,  accuracy 0.6571428775787354
Finished epoch 2,  accuracy 0.34285715222358704
Finished epoch 3,  accuracy 0.6571428775787354
Finished epoch 4,  accuracy 0.3523809611797333
Finished epoch 5,  accuracy 0.34285715222358704
Finished epoch 6,  accuracy 0.3619047701358795
Finished epoch 7,  accuracy 0.37142857909202576
Finished epoch 8,  accuracy 0.37142857909202576
Finished epoch 9,  accuracy 0.39047619700431824
Finished epoch 10,  accuracy 0.39047619700431824
Finished epoch 11,  accuracy 0.39047619700431824
Finished epoch 12,  accuracy 0.39047619700431824
Finished epoch 13,  accuracy 0.4095238149166107
Finished epoch 14,  accuracy 0.4285714328289032
Finished epoch 15,  accuracy 0.39047619700431824
Finished epoch 16,  accuracy 0.4285714328289032
Finished epoch 17,  accuracy 0.4285714328289032
Finished epoch 18,  accuracy 0.4285714328289032
Finished epoch 19,  accuracy 0.419047623872

In [94]:
accuracy = evaluate(model, X_test, y_test)
print(accuracy)

tensor(0.6952)


## Save/load model and evaluate 

In [95]:
torch.save(model, 'credit_risk_model')

In [97]:
model = torch.load('credit_risk_model')
model.eval()
accuracy = evaluate(model, X_test, y_test)
print(accuracy)

tensor(0.6952)
